In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./3.part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(f'''https://www.1aauto.com/catalog/product/fit?skuId={input_.loc[a, 'Part Code']}&pathName={input_.loc[a, 'Url'].removeprefix('https://www.1aauto.com').replace('/', '%2F')}''',
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_th = [text.strip() for text in html.xpath('//table[@class="fitTable sortableTable onea-table"]/thead/tr/th/text()')]

                # = = = = = = = = = = = = = = =

                list_td = [[td.strip() for td in tr.xpath('./td/text()')] for tr in html.xpath('//table[@class="fitTable sortableTable onea-table"]/tbody/tr')]

                # = = = = = = = = = = = = = = =

                list_oem = [text.replace(' ', '') for text in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                        'Part Number': input_.loc[a, 'Part Number'],
                                        'OEM': oem,
                                        'Row': [i+1 for i in range(len(list_td))]})

                for i, td in zip(range(len(list_td)), list_td):
                    for j, th in zip(range(len(list_th)), list_th):
                        df_temp.loc[i, th] = td[j]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./4.vehicle_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./vehicle_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：  1] - 388.https://www.1aauto.com/chevrolet-gmc-front-window-glass-run-channel-fairchild-automotive-g1008/i/1awsd00556
[剩余数量：1000] - [当前时间：17:46:47]

[状态：ok，尝试次数：  1] - 395.https://www.1aauto.com/1967-70-ford-f100-f250-f350-6-piece-window-sweep-and-run-channel-weatherstrip-seal-set-fairchild-automotive-kf1003-6/i/1awss00125
[剩余数量：999] - [当前时间：17:46:48]

[状态：ok，尝试次数：  1] - 396.https://www.1aauto.com/ford-bronco-ii-ranger-window-sweep-4-piece-set-fairchild-automotive-kf2025/i/1awss00098
[剩余数量：998] - [当前时间：17:46:48]

[状态：ok，尝试次数：  1] - 379.https://www.1aauto.com/1982-92-chevrolet-camaro-pontiac-firebird-front-driver-side-triangle-weatherstrip-seal-fairchild-automotive-g4023/i/1awsx00020
[剩余数量：997] - [当前时间：17:46:48]

[状态：ok，尝试次数：  1] - 397.https://www.1aauto.com/1982-92-chevrolet-camaro-pontiac-firebird-5-piece-door-roofrail-and-trunk-seal-kit-diy-solutions-wss00487/i/1awsk00106
[剩余数量：996] - [当前时间：17:46:48]

[状态：ok，尝试次数：  1] - 400.https://www.1aauto.com/chevrolet-gmc-front-and-